<a href="https://colab.research.google.com/github/shrikantnarayankar15/hourse_rent_insaid/blob/master/hourse_rent_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EDA on House Rent


##1.Problem Statement
  here we are going to observe the house prices by its featues like ground_size, bedroom_number, garage have or not, neightbourhood, quality of residence, etc. and we will use different models to predict the output of houses i.e. house price.

##2.Importing Packages

In [0]:
import pandas as pd
import numpy as np
from sympy.utilities.iterables import multiset_permutations
import matplotlib.pyplot as plt
from scipy.stats import boxcox_normmax, zscore
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.ensemble import GradientBoostingRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.preprocessing import RobustScaler
from scipy.stats import boxcox_normmax, zscore
from multiprocessing import cpu_count
from lightgbm import LGBMRegressor
from scipy.special import boxcox1p
import matplotlib.pyplot as plt

from sklearn.svm import SVR
import pandas as pd
import numpy as np
#library
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import random as rnd

from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

# Importing Models
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Importing other tools
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, classification_report, make_scorer
from sklearn.metrics import accuracy_score, recall_score, precision_recall_curve
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

##3.Data Preprocessing

In [0]:
#Loading Data
df = pd.read_csv('https://raw.githubusercontent.com/shrikantnarayankar15/hourse_rent_insaid/master/train.csv',index_col=0)
test_data = pd.read_csv('https://raw.githubusercontent.com/shrikantnarayankar15/hourse_rent_insaid/master/test.csv', index_col=0)

In [0]:
  df.head()

###1.Removing Outliers

In [0]:
sns.scatterplot(x="GrLivArea", y="SalePrice", data=df)

In [0]:
df = df[~(df.GrLivArea>4500)]

In [0]:
sns.scatterplot(x="GrLivArea", y="SalePrice", data=df)

###2.Combining test and training dataset
**test** dataset does not have independent variable i.e.output variable thats why while combining we are concatenating train dataset with test dataset without SalePrice feature.

In [0]:
X = pd.concat([df.drop("SalePrice", axis=1), test_data])

In [0]:
X.head()

df.head()

###3.data Description


*   Total 1460 rows are there
*   81 columns are there





In [0]:
X.describe()

In [0]:
# from  pandas_profiling import ProfileReport
# profile = ProfileReport(test_data, title='Pandas Profiling Report', html={'style':{'full_width':True}})
# # profile.to_notebook_iframe()
# profile.to_file(output_file="output.html")

###4.Checking Null Values

####1.Columns with high number of NaN

In [0]:
# missing values
w = X.isnull().sum()
w = w[w>0]
w.sort_values(ascending=False)[:5]

In [0]:
X.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'], axis=1, inplace=True)

we will drop this columns as they high nnumber of Null values


####2.filling null values
we Are filling null values with Neighborhood, we selected this feature to fill null values because area wise they have high chances of probablity that they may have same feature.

In [0]:
# LotFrontage and MasVnrArea are continous variables
conts_cols = ['LotFrontage', 'MasVnrArea', 'BsmtFullBath','BsmtHalfBath','TotalBsmtSF','BsmtUnfSF','BsmtFinSF2','BsmtFinSF1','GarageArea','GarageCars',]
X[conts_cols] = X.groupby("Neighborhood")[conts_cols].transform(lambda x: x.fillna(x.median()))

In [0]:
# fill the categorical variables
conts_cols = ['GarageCond','MSZoning', 'Functional','SaleType', 'KitchenQual',
       'Exterior2nd', 'Exterior1st','Utilities','GarageYrBlt','GarageFinish','GarageQual','GarageType','BsmtFinType2','BsmtExposure','BsmtFinType1','BsmtCond','BsmtQual','MasVnrType','Electrical']
X[conts_cols] = X.groupby("Neighborhood")[conts_cols].transform(lambda x: x.fillna(x.mode()[0]))

In [0]:
#check null values
w = X.isnull().sum()
w = w[w>0]
w.sort_values(ascending=False)

###5.Combining features


*   We can make ground living area and basement area together into TotalSF
*   Same we will combine openporch, enclosedd,3Ssn and screen because they are all porch areas.
*   TotalBath will combile all the bathrooms in the house


In [0]:
df["TotalSF"] = df["GrLivArea"] + df["TotalBsmtSF"]
df["TotalPorchSF"] = df["OpenPorchSF"] + df["EnclosedPorch"] + df["3SsnPorch"] + df["ScreenPorch"]
df["TotalBath"] = df["FullBath"] + df["BsmtFullBath"] + 0.5 * (df["BsmtHalfBath"] + df["HalfBath"])

In [0]:
df.drop(['BsmtFinSF2', 'MiscVal', 'PoolArea','LowQualFinSF'], axis=1,inplace=True)

In [0]:
X["TotalSF"] = X["GrLivArea"] + X["TotalBsmtSF"]
X["TotalPorchSF"] = X["OpenPorchSF"] + X["EnclosedPorch"] + X["3SsnPorch"] + X["ScreenPorch"]
X["TotalBath"] = X["FullBath"] + X["BsmtFullBath"] + 0.5 * (X["BsmtHalfBath"] + X["HalfBath"])

In [0]:
X.drop(['GrLivArea', 'TotalBsmtSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','FullBath','BsmtFullBath','BsmtHalfBath','HalfBath'],axis=1,inplace=True)

###6.Checking the 0 values

In [0]:
(X==0).sum().nlargest(7)

As we can see PoolArea, LowQualFinSFm,MiscVal and BsmtFinSF2 have more zeros, we are removing them

In [0]:
X.drop(['BsmtFinSF2', 'MiscVal', 'PoolArea','LowQualFinSF'], axis=1,inplace=True)

##4.EDA on house-price

###1.checking counts of fireplaces,basements


In [0]:
df['fireplace_Y_N'] = df['Fireplaces']>1
plt.figure(figsize=(20,10))
sns.countplot( x='fireplace_Y_N',data=df)

- Many houses dont have fireplaces

In [0]:
plt.figure(figsize=(10,10))
df['TotalBsmtSF_Y_N'] = df['TotalBsmtSF']>1
sns.countplot( x='TotalBsmtSF_Y_N',data=df)

- only few houses dont have any kind of basement.

In [0]:
plt.figure(figsize=(10,6))
sns.countplot( x='SaleCondition',data=df)

- normal means this are the house which are already build. and they have higher amout of data in our dataset.

###2.With Saleprice


In [0]:
sns.scatterplot( x='GrLivArea', y='SalePrice',hue='fireplace_Y_N',data=df)

In [0]:
df[['GrLivArea','SalePrice']].corr()

- As Ground living area increases saleprice will increase too.
- houses with fireplaces have more price

In [0]:
# TotalSF
sns.barplot(x="TotalBath", y="SalePrice", data=df)

In [0]:
#Type of house and price
df.groupby(['MSSubClass'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

from here we can say the type<br>
60-   2-STORY 1946 & NEWERare having highest saleprice<br>
after that 120, 75, 20 and so on 

In [0]:
#Type of house and price
df.groupby(['MSZoning'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

-Floating Village Residential have higher prices than other zoning areas


Floating Village Residential are having higher sale price <br>
commercial are having cheaper rates

In [0]:
fig, axs = plt.subplots(1,2,figsize=(20,10))
axs = axs.ravel(order='F')
sns.scatterplot(x='LotFrontage', y="SalePrice", data=df,ax=axs[0])
sns.scatterplot(x='LotArea', y="SalePrice", data=df, ax=axs[1])


In [0]:
df.loc[:,['LotFrontage','LotArea','SalePrice']].corr()

From here we can say if lotarea and lotfrontage will increase sale price will increase too

In [0]:
df.groupby(['Street'])['SalePrice'].mean().plot.bar()

Home having street as Paved have more sale price 

In [0]:
df.groupby(['LotShape'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()


In [0]:
df.groupby(['LandContour'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

**"Hillside - Significant slope from side to side"** have higher saleprice

In [0]:
df.groupby(['Utilities'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

homes with all public utilites have higher saleprice


In [0]:
df.groupby(['LotConfig'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

home which is at the end of stree have higher saleprice


In [0]:
df.groupby(['LandSlope'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

not that much differencce between sev and mode. Gentle slope house have little decrease in saleprice

In [0]:
df.groupby(['Neighborhood'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

Neighbourhood with Norigdge have higher saleprices.

In [0]:
df.groupby(['BldgType'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

In [0]:
df.groupby(['OverallQual'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()


In [0]:
df.groupby(['RoofStyle'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

In [0]:
df.groupby(['RoofMatl'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

In [0]:
df.groupby(['BsmtFullBath'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

In [0]:
df.columns

In [0]:
df.groupby(['Condition1'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

In [0]:
df.groupby(['Condition2'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

In [0]:

bins = np.linspace(1900,2010,12).astype(int)
df['YearBuilt'] = pd.cut(df['YearBuilt'], bins=bins)

In [0]:
df.groupby(['YearBuilt'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

2000-2010 have higher prices. this is due to the inflation at every year

In [0]:
a = np.array(['1872','1899']).astype(int)
b = np.linspace(1950,2010,7).astype(int)
bins = np.concatenate([a,b])
df['YearRemodAdd'] = pd.cut(df['YearRemodAdd'], bins=bins)

In [0]:
df.groupby(['YearRemodAdd'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

In [0]:
df.groupby(['RoofStyle'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

In [0]:
X['RoofStyle'].value_counts()

In [0]:
df.groupby(['RoofMatl'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

In [0]:

df.groupby(['Foundation'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

Roof material with **Wood Shingles** have higher prices.

In [0]:
df.groupby(['Electrical'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

house with electrical circuit as SBrkr have higher prices.

In [0]:
df.groupby(['SaleCondition'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

if sale condition is partial have higher sale prices. because they are to be build in future and will be recorded as new homes.

###3.With OverallQuality

In [0]:
sns.barplot(x='MSSubClass', y='OverallQual',data=df)

MSsubclass with 60 and 120 have good overallQuality<br>
60-(2-STORY 1946 & NEWER)<br>
120-(1-STORY PUD (Planned Unit Development) - 1946 & NEWER

In [0]:
plt.figure(figsize=(20,10))
sns.barplot(x='Neighborhood', y='OverallQual',data=df, order=df.groupby('Neighborhood')['OverallQual'].mean().sort_values().index)

-NridgHt have higher quality <br>
-meaddow have lowest quality of houses

In [0]:
plt.figure(figsize=(20,10))
sns.barplot(x='MSZoning', y='OverallQual',data=df, order=df.groupby('MSZoning')['OverallQual'].mean().sort_values().index)

-FV have higher overallquality in MSZoning

In [0]:
plt.figure(figsize=(20,10))
sns.lineplot(x='OverallQual', y='SalePrice',data=df)

We can say from this as the OverallQual increases, price of house also increase.

- Names have more houses after that CollgCr is there.
- Blueste have lowest no. of house

###4.Data Distribution using box plot

In [0]:
 numeric_columns = df.select_dtypes(include=['number']).columns

In [0]:
df[numeric_columns]

In [0]:
df.dtypes.unique()

In [0]:
#standardize 
from sklearn.preprocessing import StandardScaler
df_copy = df[numeric_columns].copy()
df_copy[numeric_columns] = StandardScaler().fit_transform(df[numeric_columns])

In [0]:
plt.figure(figsize=(20,10))
ax=sns.boxplot(data=df_copy[numeric_columns], orient="h", palette="Set2")

###5.FeatureCorrelation

In [0]:
#dropping the columns after eda
df.drop(['GrLivArea', 'TotalBsmtSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','FullBath','BsmtFullBath','BsmtHalfBath','HalfBath'],axis=1,inplace=True)

In [0]:

features = df.select_dtypes(include=('int64','float64')).columns
features = np.array(features)
index = np.argwhere(features=='SalePrice')
features = np.delete(features, index)


In [0]:
w = pd.get_dummies(df[features])
w = w.fillna(0)
x_corr = w
y_corr = np.log(df['SalePrice'])

In [0]:
plt.figure(figsize=(20,10))
from yellowbrick.target import FeatureCorrelation
visualizer = FeatureCorrelation(labels=features)
visualizer.fit(x_corr.values, y_corr.values)

##5.Preprocessing after EDA


*   creating bins for yearbuilt
*   same for yearremoveadd



In [0]:
a = np.array(['1872','1899']).astype(int)
b = np.linspace(1900,2010,12).astype(int)
bins = np.concatenate([a,b])

X['YearBuilt'] = pd.cut(X['YearBuilt'], bins=bins)

In [0]:
# a = np.array(['1872','1899']).astype(int)
b = np.linspace(1950,2010,7).astype(int)
bins = np.concatenate([a,b])
X['YearRemodAdd'] = pd.cut(X['YearRemodAdd'], bins=bins)

##6.Model Evaluation

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
np.random.seed(123)

###1.Convert Categorical Variables to numerical data



In [0]:
X_p = pd.get_dummies(X)
train_data= X_p.loc[df.index]
train_temp= df['SalePrice']
train_output = np.log(df['SalePrice'])

In [0]:
X_p.shape

In [0]:
residuals = train_output - LinearRegression().fit(train_data, train_output).predict(train_data)
outliers = residuals[np.abs(zscore(residuals)) > 3].index
train_data = train_data.drop(outliers)
train_output = train_output.drop(outliers)
train_temp = train_temp.drop(outliers)

In [0]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import ElasticNet
logi = ElasticNet()
sfs1 = SFS(logi, 
           k_features=245, 
           forward=False, 
           floating=False, 
           verbose=2,
           n_jobs=5,
           scoring='neg_mean_squared_error',
           cv=0)
sfs1 = sfs1.fit(train_data.values, train_temp)

In [0]:
# from sklearn.feature_selection import SelectKBest, chi2
# model_kbest = SelectKBest(chi2, k=260).fit(train_data, train_temp)
train_data.head()

In [0]:
train_data=train_data.iloc[:,list(sfs1.k_feature_idx_)]

In [0]:
train_data.shape,train_output.shape

###2.Split the Dataset into train and test

In [0]:
X_th, X_tr, y_th,y_tr = train_test_split(train_data, train_output, test_size=0.2)

In [0]:
X_th.shape, X_tr.shape, y_th.shape,y_tr.shape

###3.Standardize the Data

In [0]:
from sklearn.preprocessing import MinMaxScaler
X_th = StandardScaler().fit_transform(X_th)
X_tr = StandardScaler().fit_transform(X_tr)

###4.Predict Using Different Models


1.   Linear Regression
2.   Decision Tree
3.   Random Forest
4.   GRIDCV



#####1.Linear Regression

In [0]:
from sklearn.tree import DecisionTreeRegressor
#create model
model = LinearRegression()
#fit data into model i.e.train
model.fit(X_th, y_th)

In [0]:
#predict the test data
y_pred = model.predict(X_tr)

In [0]:
from sklearn.metrics import mean_squared_error
from math import sqrt
#calculate the accuracy
rms_linear = sqrt(mean_squared_error(y_tr, y_pred))
print('LinearRegression:',rms_linear)

#####2.Decision Tree

In [0]:
from sklearn.tree import DecisionTreeRegressor
#create model
model = DecisionTreeRegressor()
#fit data into model i.e.train
model.fit(X_th, y_th)

In [0]:
#predict the test data
y_pred = model.predict(X_tr)

In [0]:
#calculate the accuracy
rms_decisiontree = sqrt(mean_squared_error(y_tr, y_pred))
print('DecisionTreeRegressor:',rms_decisiontree)

#####3.Random Forest

In [0]:
from sklearn.ensemble import RandomForestRegressor
random_forest_model = RandomForestRegressor(max_depth=2, random_state=0)
random_forest_model.fit(X_th, y_th)

In [0]:
#predict the test data
y_pred = random_forest_model.predict(X_tr)

In [0]:
#calculate the accuracy
rms_random = sqrt(mean_squared_error(y_tr, y_pred))
print('DecisionTreeRegressor:',rms_random)

#####4.GRID CV

In [0]:

from sklearn.model_selection import GridSearchCV
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
grid_search = GridSearchCV(estimator = random_forest_model, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_th, y_th)

In [0]:
#predict the test data
y_pred = grid_search.predict(X_tr)

In [0]:
#calculate the accuracy
rms_gridcv = sqrt(mean_squared_error(y_tr, y_pred))
print('GRIDCV:',rms_gridcv)

#####5.RandomizedSearchCV

In [0]:

from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, chi2
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
X_new = SelectKBest(chi2, k=20).fit_transform(X, y)
rf_random = RandomizedSearchCV(estimator = random_forest_model, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_th, y_th)

In [0]:
#predict the test data
y_pred = rf_random.predict(X_tr)

In [0]:
#calculate the accuracy
rms_randomcv = sqrt(mean_squared_error(y_tr, y_pred))
print('GRIDCV:',rms_randomcv)

#7.Kaggle Competition testing

In [0]:
def train_ensemble(X_th, y_th):
  from sklearn.pipeline import make_pipeline
  from xgboost import XGBRegressor
  from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
  kfolds = KFold(n_splits=10, shuffle=True, random_state=42)
  alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
  alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
  e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
  e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

  ridge = make_pipeline(RobustScaler(),
                        RidgeCV(alphas=alphas_alt, cv=kfolds,))

  lasso = make_pipeline(RobustScaler(),
                        LassoCV(max_iter=1e7, alphas=alphas2,
                                random_state=42, cv=kfolds))

  elasticnet = make_pipeline(RobustScaler(),
                            ElasticNetCV(max_iter=1e7, alphas=e_alphas,
                                          cv=kfolds, random_state=42, l1_ratio=e_l1ratio))
                                          
  svr = make_pipeline(RobustScaler(),
                        SVR(C= 20, epsilon= 0.008, gamma=0.0003,))


  gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                    max_depth=4, max_features='sqrt',
                                    min_samples_leaf=15, min_samples_split=10, 
                                    loss='huber', random_state =42)
                                    

  lightgbm = LGBMRegressor(objective='regression', 
                                        num_leaves=4,
                                        learning_rate=0.01, 
                                        n_estimators=5000,
                                        max_bin=200, 
                                        bagging_fraction=0.75,
                                        bagging_freq=5, 
                                        bagging_seed=7,
                                        feature_fraction=0.2,
                                        feature_fraction_seed=7,
                                        verbose=-1,
                                        #min_data_in_leaf=2,
                                        #min_sum_hessian_in_leaf=11
                                        )
                                        

  xgboost = XGBRegressor(learning_rate=0.01, n_estimators=3460,
                                      max_depth=3, min_child_weight=0,
                                      gamma=0, subsample=0.7,
                                      colsample_bytree=0.7,
                                      objective ='reg:squarederror', nthread=-1,
                                      scale_pos_weight=1, seed=27,
                                      reg_alpha=0.00006, random_state=42)

  # stack
  stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet,
                                              gbr, xgboost, lightgbm),
                                  meta_regressor=xgboost,
                                  use_features_in_secondary=True)
  print("hehe")
  stack_gen.fit(X_th, y_th)
  return stack_gen

In [0]:
X_test = X_p.loc[test_data.index]
X_test = X_test.iloc[:,list(sfs1.k_feature_idx_)]
X_test = StandardScaler().fit_transform(X_test)

In [0]:
train_data= StandardScaler().fit_transform(train_data)


In [0]:
# have train_data and train_output
model = train_ensemble(train_data,train_output.values)

In [0]:
y_pred=model.predict(X_test)
submission = pd.DataFrame({"Id": test_data.index.values, "SalePrice":np.exp(y_pred)})
submission.to_csv("submission_house_rent.csv", index=False)